In [4]:
#Initialize libraries

import praw
import pandas as pd
import emoji
from prawcore.exceptions import TooManyRequests
import re
import time

In [5]:
# Retry function with exponential backoff for network-related errors
def retry_with_backoff(retries, delay, func, *args, **kwargs):
    attempt = 0
    while attempt < retries:
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {str(e)}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2  # Exponential backoff
            attempt += 1
    raise Exception("Maximum retry attempts reached")

# Initialize Reddit API credentials
user_agent = "Scraper 1.0 by /u/NewBreadfruit7592"
reddit = praw.Reddit(
    client_id="V8UWamg3F7jKCy3GI7ojtA",
    client_secret="nTcJKHEILYlGbHlZC00I78J_a9TLPw",
    user_agent=user_agent
)

# Define search query parameters
keywords = ['beks','pucha', 'paps', 'squammy', 'abangers']  # Add more keywords as needed
query = ' OR '.join(keywords)

# Initialize list for content data
content_data = []

# Helper function to remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Fetch submissions using search query, limit to 100 posts
for submission in reddit.subreddit('Philippines').search(query, limit=1000):
    try:
        # Combine title and body of the post
        combined_text = f'{submission.title}\n{submission.selftext}'

        # Remove emojis from the post content
        combined_text_no_emoji = remove_emoji(combined_text)

        # Check if specific keyword(s) appear in the post
        if any(keyword in combined_text_no_emoji.lower() for keyword in keywords):
            # Add the post content to the data
            content_data.append(combined_text_no_emoji)

        # Fetch all comments from the post using retry with backoff
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            # Remove emojis from the comment
            comment_no_emoji = remove_emoji(comment.body)

            # Check if specific keyword(s) appear in the comment
            if any(keyword in comment_no_emoji.lower() for keyword in keywords):
                content_data.append(comment_no_emoji)
    except Exception as e:
        print(f"Error while processing post: {str(e)}")

# Print number of content entries found
print(f"Number of posts and comments collected: {len(content_data)}")

# Convert the data to a DataFrame
df_new = pd.DataFrame(content_data, columns=["Content"])


Number of posts and comments collected: 295


In [6]:
# Save the data to CSV
file_path = 'bekspuchapapssquammyabangers.csv'
df_new.to_csv(file_path, index=False, encoding='utf-8')

print(f"Data saved to {file_path}")

Data saved to Newdata_noemoji.csv
